In [ ]:
# Imports
import pycryptodome
from Crypto.Cipher import AES
from Crypto.Hash import HMAC, SHA256
from Crypto.Random import get_random_bytes

In [ ]:
# Helper functions
## keyToeN(string) -> tuple
## Takes in the path to a file containing a
## public RSA key, and returns a tuple with e
## and N values.
## Intended for use with rsaEncrypt(), to be
## piped into the key parameter in tuple form,
## or users can use the string form and just
## pass the file path to the function. 
def pubKeyToeN(file: str | None = None, key: str | int | None = None):
    if (file == None) and (key == None):
        raise ValueError("Either file or key must be provided")
    if (file != None) and (key != None):
        raise ValueError("Only one of file or key can be provided")
    #TODO: Figure out how the hell to go from long ahh key to a and N

def privKeyTodN(file: str | None = None, key: str | int | None = None):
    if (file == None) and (key == None):
        raise ValueError("Either file or key must be provided")
    if (file != None) and (key != None):
        raise ValueError("Only one of file or key can be provided")
    #TODO: idfk

In [ ]:
def rsaEncrypt(plaintext, key: tuple | str | int):
    if type(key) == tuple:
        ciphertext = (plaintext ** key[0]) % key[1]
    if (type(key) == str) or (type(key) == int):
        ciphertext = (plaintext ** pubKeyToeN(key)[0]) % pubKeyToeN(key)[1]
    if (type(key) != str) and (type(key) != int) and (type(key) != tuple):
        raise TypeError("Invalid key type")
    return ciphertext

def rsaDecrypt(ciphertext, key: tuple | str | int):
    if type(key) == tuple:
        plaintext = (ciphertext ** key[0]) % key[1]
    if (type(key) == str) or (type(key) == int):
        plaintext = (ciphertext ** privKeyTodN(key)[0]) % privKeyTodN(key)[1]
    if (type(key) != str) and (type(key) != int) and (type(key) != tuple):
        raise TypeError("Invalid key type")
    return ciphertext


In [ ]:
def rsaBruteForce():
    ...